## 2.1 Processes ##
1. Process: an abstraction of a running program.

2. Pseudoparallelism v.s. multiprocessor 

3. In each CPU, there is always one process running.

### 2.1.1 The Process Model ###
1. In this model, all the runnable software on the computer, sometimes including the operating system, is organized into a number of ***sequential processes(processes)***.

2. A process is an instance of an executing program, including the current values of the program counter, registers, and variables. Conceptually, each process has its own virtual CPU. 

3. Processes must not be programmed with built-in assumptions about timing.

4. Process v.s. program
        - a ***process*** is an activity of some kind. It has a program, input, output, and a state.
        - a single processor may be shared among several processes, with some scheduling algorithm being accustomed to determine when to stop work on one proces and service a different one.
        - in contrast, a ***program*** is something that may be stored on disk, not doing anything.

### 2.1.2 Process Creation ###
1. Four principle events cause processes to e created:
        - System initialization.
        - Execution of a process-creation system call by a running process.
        - A user request to create a new process.
        - Initiaiton of a batch job.
        
2. Deamons: processes that stey in the background to handle some activity.

3. Technically, in all cases, a new process is created by having an existing process execute a process creation system call.

4. `fork` in the only one system call to create a new process in UNIX. This call creates an exact clone fo the calling process. After that, the child process executes `execve` or a similar system call to change its memory image and run a new program.

5. In Windows, in contrast, s single Win32 function call, `CreateProcess`, handles both process creation and loading the correct program into the new process.

6. In both UNIX and Windows systems, after a process is created, the parent and child have their own distinct address spaces. NO WRITABLE MEMORY IS SHARED.


### 2.1.3 Process Termination ###
1. Conditions where new process will terminate:
        - Normal exit (voluntary). E.g. the process has done its work
        - Error exit (voluntary). E.g. the process discovers a fatal error
        - Fatal error (involuntary). E.g. caused by the process, bugs
        - Killed by another process (involuntary). E.g. (by system call `kill`)
        
### 2.1.4 Process Hierarchies ###
1. In some systems, when a process creates another process, the parent process and child process continue to be associated in certain ways. The child process can itself create more processes, forming a process hierarchy.

2. A process has only one parent but can have multiple chilren.

3. In UNIX, a process and all of its children and further descendants together form a process group.

4. Unlike in Windows, process in UNIX cannot disinherit their children.

### 2.1.5 Process States ###
1. Three states a process may be in:
        - Running (actually using the CPU at that instant).
        - Ready (runnable; temporarily stopped to let another process run).
        - Blocked (unable to run until some external event happends).

2. Four possible transitions among these three states:
        - Process blocks for input.
        - Scheduler picks another paroacess.
        - Scheduler picks this process.
        - Input becomes available.
    
3. The lowest layer of a process-structured operating system handles interrupts and scheuling. Above taht layer are sequential processes.

### 2.1.6 Implementation of Processes ###
1. The operating system maintains a table (an array of structures), called the **process table/process control blocks**. This entry contains import information abotu the process' state, i.e., its program counter, stack pointer, memory allocation, the status of its open files, its accouting and scheduling information.

2. Associated with each I/O class is a location (typically at a fixed location near the bottom of memory) called the **interrupt vector**. It contains the address of the interrupt service procedure. 

### 2.1.7 Modeling Multiprogramming ###
1. Skeleton of what the lowest level of the operating system does when an interrupt occurs:
        a. Hadware stacks program counter, etc.
        b. Hardware loads new program counter from interrupt vector.
        c. Assembly-language procedure saves registers.
        d. Assembly-language procedure sets up new stack.
        e. C interrupt service runs (typically reads and buffers input)
        f. Scheduler decides which process is to run next.
        g. C procedure returns to the assembly code.
        h. Assembly-language procedure starts up new current process.
        
2. Degree of multiprogramming : don't quite get this 